In [1]:
import requests
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Read in and Aggregate County Data

In [2]:
ia12 = pd.read_excel("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/raw_data/Iowa_2012_cds.xlsx")

In [3]:
ia12 = ia12["County District Population"].str.split(' ', expand=True)

In [4]:
ia12.columns = ["county","district_112","pop"]

In [5]:
ia12 = ia12[["county","district_112"]]

In [6]:
ia12.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/iowa/iowa_2012.csv", index = False)

After manually imputing county names via the published map (thanks IA leg), I have a csv of the states districts

In [7]:
ia_cts = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/raw_data/Iowa_2022.csv")

In [8]:
#remove the underscore used to prevent deliniation
ia_cts["county"] = ia_cts["county"].str.replace("_"," ")

In [9]:
fips = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/raw_data/iowa_fips.csv")
fips["county"] = fips["county"].replace("O Brien","O'Brien")

In [10]:
ia_cts = pd.merge(ia_cts,fips, on="county", how="outer")

In [11]:
ia_cts["ct_FIPS"] = ia_cts["FIPS"].astype(str).str[2:]

# Read in and Aggregate District Data

In [12]:
d = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/aggregated_fairness.csv")
#s = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/2022_redraw_workbook.csv")

In [13]:
d = d[d["ST"] == "IA"]
d = d[d["year"] > 2018]
#s = s[s["ST"] == "IA"]

In [14]:
data_118 = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/full_districts/data_118.csv")
data_117 = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/full_districts/data_117.csv")

In [15]:
data_118 = data_118[data_118["ST"] == "IA"]
data_117 = data_117[data_117["ST"] == "IA"]
dist = pd.concat([data_118,data_117])
dist["district"] = dist["ST#"].str[-1]
dist

,year,congress,ST,ST#,PVI,metric,seat,is_GOP,district
103,2022,118,IA,IA1,R+4,0.52,NaN,NaN,1
104,2022,118,IA,IA2,R+6,0.53,NaN,NaN,2
105,2022,118,IA,IA3,R+2,0.51,NaN,NaN,3
106,2022,118,IA,IA4,R+27,0.64,NaN,NaN,4
159,2020,117,IA,IA1,R+4,0.54,R,1.0,1
160,2020,117,IA,IA2,R+4,0.54,R,1.0,2
161,2020,117,IA,IA3,R+3,0.53,D,0.0,3
162,2020,117,IA,IA4,R+16,0.66,R,1.0,4


In [16]:
dicts = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/metric_converter.csv")
dists = pd.merge(dist,dicts, on="metric")

In [17]:
dists = dists.sort_values("year").sort_values("district").reset_index(drop=True)
dists

,year,congress,ST,ST#,PVI,metric,seat,is_GOP,district,pvi_range,prob_GOP
0,2020,117,IA,IA1,R+4,0.54,R,1.0,1,R+4,0.788991
1,2022,118,IA,IA1,R+4,0.52,NaN,NaN,1,R+2,0.653846
2,2020,117,IA,IA2,R+4,0.54,R,1.0,2,R+4,0.788991
3,2022,118,IA,IA2,R+6,0.53,NaN,NaN,2,R+3,0.688172
4,2020,117,IA,IA3,R+3,0.53,D,0.0,3,R+3,0.688172
5,2022,118,IA,IA3,R+2,0.51,NaN,NaN,3,R+1,0.591837
6,2020,117,IA,IA4,R+16,0.66,R,1.0,4,R+8 to R+20,0.953509
7,2022,118,IA,IA4,R+27,0.64,NaN,NaN,4,R+8 to R+20,0.953509


In [19]:
ia_cts.head()

,county,district_112,district_118,FIPS,state,ct_FIPS
0,Allamakee,1,2,19005,IA,005
1,Benton,1,2,19011,IA,011
2,Black Hawk,1,2,19013,IA,013
3,Bremer,1,2,19017,IA,017
4,Buchanan,1,2,19019,IA,019
